In [1]:
import numpy as np
import sys
import time
import h5py as h5
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation

sys.path.append(r"C:\Users\haoyuan\Documents\GitHub\CrystalDiff")

from CrystalDiff import util, pulse, lclsutil, crystal, lightpath

# Get the Crystal Geometries

In [2]:
# ----------------------------------------------------------------------------------------------------------
#                       Step 1: Pulse
# ----------------------------------------------------------------------------------------------------------
energy_center = 10.
pre_length = 1e6

# Set up the pulse
my_pulse = pulse.GaussianPulse3D()
my_pulse.set_pulse_properties(central_energy=energy_center,
                              polar=[0., 1., 0.],
                              sigma_x=708.2581446128465,
                              sigma_y=708.2581446128465,
                              sigma_z=1.,
                              x0=np.array([0., 0., -pre_length]))

# ----------------------------------------------------------------------------------------------------------
#                       Step 2: Split
# ----------------------------------------------------------------------------------------------------------
dtheta = np.arctan(1.5 * 1e-3 / 5.)  # This is the deviation angle.
half_period = util.get_grating_period(dtheta=dtheta,
                                      klen_in=my_pulse.klen0) / 2

grating_list = [crystal.RectangleGrating(), crystal.RectangleGrating()]
grating_list[0].set_a(half_period)
grating_list[0].set_b(half_period)
grating_list[1].set_a(half_period)
grating_list[1].set_b(half_period)

# ----------------------------------------------------------------------------------------------------------
#                       Step 3: Delay Lines
# ----------------------------------------------------------------------------------------------------------
# Some meta data for the delay line.
h_length = 2. * np.pi / (1.9201 * 1e-4)

# Some crystal properties
chi0 = complex(-0.97631E-05, 0.14871E-06)
chih_sigma = complex(0.59310E-05, -0.14320E-06)
chihbar_sigma = complex(0.59310E-05, -0.14320E-06)
chih_pi = complex(0.46945E-05, -0.11201E-06)
chihbar_pi = complex(0.46945E-05, -0.11201E-06)

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#                       Crystal for branch  1
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Set up the angles
angle_offset_1 = 0 # dtheta

# Bragg angle
bragg = np.radians(18.836) + 13e-6
miscut = np.deg2rad(10.)

# Get crystal angles
eta_b1 = np.zeros(6, dtype=np.float64)
theta_b1 = np.array([np.pi / 2. + bragg,
                     3 * np.pi / 2. + bragg,
                     3 * np.pi / 2. - bragg,
                     np.pi / 2. - bragg,
                     3 * np.pi / 2. - bragg,
                     np.pi / 2. - bragg])
rho_b1 = theta_b1 + np.pi
#rho_b1[1] += np.deg2rad(10)
tau_b1 = np.array([0., 0, 0, 0.,
                   0, 0.])

# Add the effect of the global rotation due to the grating momentum transfer
theta_b1 += angle_offset_1
rho_b1 += angle_offset_1

# Initialize the crystals
crystal_list_1 = lclsutil.get_crystal_list(num=6,
                                           hlen=np.array([h_length, ] * 6),
                                           theta=theta_b1,
                                           eta=eta_b1,
                                           rho=rho_b1,
                                           tau=tau_b1,
                                           chi0=chi0,
                                           chih_sigma=chih_sigma,
                                           chihbar_sigma=chihbar_sigma,
                                           chih_pi=chih_pi,
                                           chihbar_pi=chihbar_pi)

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#                       Crystal for branch  2
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Set up the angles
angle_offset_2 = - dtheta
bragg = np.radians(18.836) + 13e-6

# Get crystal angles
eta_b2 = np.zeros(4, dtype=np.float64)
theta_b2 = np.array([3 * np.pi / 2. - bragg,
                     np.pi / 2. - bragg,
                     np.pi / 2. + bragg,
                     3 * np.pi / 2. + bragg])
rho_b2 = theta_b2 + np.pi
tau_b2 = np.zeros(4, dtype=np.float64)

# Add the effect of the global rotation due to the grating momentum transfer
theta_b2 += angle_offset_2
rho_b2 += angle_offset_2

# Initialize the crystals
crystal_list_2 = lclsutil.get_crystal_list(num=4,
                                           hlen=np.array([h_length, ] * 4),
                                           theta=theta_b2,
                                           eta=eta_b2,
                                           rho=rho_b2,
                                           tau=tau_b2,
                                           chi0=chi0,
                                           chih_sigma=chih_sigma,
                                           chihbar_sigma=chihbar_sigma,
                                           chih_pi=chih_pi,
                                           chihbar_pi=chihbar_pi)

# ------------------------------------------------------
#   Define the positions
# ------------------------------------------------------
path_list_fix = [5e6, 1e5, 2e5, 1.05e5, 6e6, 1e6]
path_list_var = [5e6- 2e5, 1e5, 4e5, 2e4, 3e5, 8.5e4, 6e6, 1e6]
delay_time = 100

In [3]:
for x in crystal_list_1:
    print(x.h)

[     0.          30970.64174806 -10565.43518434]
[     0.         -30970.64174806  10565.43518434]
[     0.         -30970.64174806 -10565.43518434]
[    0.         30970.64174806 10565.43518434]
[     0.         -30970.64174806 -10565.43518434]
[    0.         30970.64174806 10565.43518434]


In [4]:
for x in crystal_list_1:
    print(x.normal)

[ 0.         -0.94644239  0.32287273]
[ 0.          0.94644239 -0.32287273]
[0.         0.94644239 0.32287273]
[ 0.         -0.94644239 -0.32287273]
[0.         0.94644239 0.32287273]
[ 0.         -0.94644239 -0.32287273]


# Get the output momentum vector

In [5]:
kin = np.copy(my_pulse.k0)

kout_1 = util.get_bragg_kout(kin=kin,
                             h = crystal_list_1[0].h,
                             normal = crystal_list_1[0].normal)

kout_2 = util.get_bragg_kout(kin=kout_1,
                             h = crystal_list_1[1].h,
                             normal = crystal_list_1[1].normal)

In [6]:
print(np.arctan((kout_2 - my_pulse.k0) / my_pulse.klen0) * 1e6)

[0.00000000e+00 2.74008311e-10 0.00000000e+00]


In [7]:
bragg

0.3287632179056519

In [8]:
rot_mat = Rotation.from_euler('x', 10., degrees=True)
mat = rot_mat.as_dcm()

kin = np.copy(my_pulse.k0)

kout_1 = util.get_bragg_kout(kin=kin,
                             h = crystal_list_1[0].h,
                             normal = crystal_list_1[0].normal)

kout_2 = util.get_bragg_kout(kin=kin,
                             h = crystal_list_1[0].h,
                             normal = mat.dot(crystal_list_1[0].normal))

In [9]:
print(np.arctan(np.linalg.norm(kout_2 - kout_1) / my_pulse.klen0) * 1e6)

10.10553335412833


In [10]:
print(2 * np.sin(bragg), h_length / my_pulse.klen0)

0.6457454652585171 0.6457173969460075


In [23]:
alpha = np.deg2rad(10)

t = 2 * np.sin(bragg)

tmp1= - t / np.sin(bragg + alpha) * np.array( [- np.cos(bragg) * np.sin(bragg - alpha),
                      np.cos(bragg) * np.cos(bragg - alpha) - np.cos(alpha)]) 
print( tmp1)

[0.19465736 0.06640611]


In [24]:
tmp1[1] -= np.sin(bragg - alpha) / np.sin(bragg + alpha)

In [25]:
np.linalg.norm(tmp1)

0.31850428341500475

In [26]:
np.sin(bragg - alpha) / np.sin(bragg + alpha)

0.3185042834150048